In [2]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%matplotlib qt

# O método de Newton

## $ \S 1 $ Descrição do método de Newton

Suponha que desejemos encontrar um zero de uma função _diferenciável_ $ f $.
Partindo de uma estimativa inicial $ x_0 $, a idéia por trás do __método de
Newton__ é substituir a função pela sua reta tangente na estimativa atual
$ x_n $ e tomar a intersecção desta reta com o eixo-$x$ como o valor da próxima
estimativa $ x_{n + 1} $.  Em símbolos, a reta tangente ao gráfico de $ f $ em
$ x_n $ é descrita pela equação:
\begin{equation*}\label{E:line}
y = f(x_n) + f'(x_n)\,(x - x_{n})\,. \tag{1}
\end{equation*}
A aproximação $ x_{n + 1} $ seguinte é obtida fazendo $ y = 0 $ e encontrando
o valor de $ x $ correspodente:
$$
\boxed{x_{n + 1} = x_n - \frac{f(x_n)}{f'(x_n)}}
$$

📝 A expressão do lado direito em \eqref{E:line} é o polinômio de Taylor de grau
$ 1 $ de $ f $ ao redor de $ x_n $, às vezes chamado nos cursos de Cálculo de
_linearização_ ou _aproximação linear_ de $ f $ aí. Portanto, sob outro ponto de
vista, completamente equivalente ao que descrevemos antes, o método de Newton
consiste em substituir a função original $ f $ pelo seu polinômio de Taylor de
grau $ 1 $ ao redor da estimativa atual e usar o zero deste como nova
aproximação para o zero de $ f $.

📝 O método de Newton também é conhecido como _método de Newton-Raphson_; Joseph
Raphson (1668–1715) foi um matemático inglês contemporâneo de Newton.

__Exemplo 1:__ Rode em seqüência o código nas três células abaixo para ver uma
animação do método de Newton aproximando um zero da função $ f(x) =  x^3 - 2x -
3 \ln x $. (Para facilitar a navegação posterior, depois de executar estas
células, você pode escondê-las.) 

In [6]:

def print_solution(xs: list[float], ys: list[float], freq: int = 1) -> None:
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first and last line are always printed. If freq == 0, then only these
          lines are printed.
    Returns: None.
    Prints: A header and the table described above.
    """
    def print_header() -> None:
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n: int, x: float, y: float) -> None:
        """
        Pretty-prints n, x and y.
        """
        print(f"|      {n:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    print_line(0, xs[0], ys[0])
    for n in range(1, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [7]:
def newton_animation(f: Callable[[float], float],
                     df: Callable[[float], float],
                     a: float, b: float, x: float,
                     N: int = 4, title: str = "", duration: float = 0.75
                     ) -> tuple[list[float], list[float]]:
    """
    Displays an animation of Newton's method applied to a function.
    Parameters:
        * A differentiable real function f.
        * Its derivative df (as a function).
        * The two endpoints a and b of the interval in the x-axis where the
          animation takes place.
        * An initial estimate x for the zero in [a, b].
        * The maximum number N of iterations.
        * A title to be displayed at the top of the diagram.
        * The duration of the pause between slides, in seconds.
          Set duration = 0 to produce a figure instead of an animation.
    Returns:
        * Two lists xs and ys containing the estimates and the values of the
          function f at each of them.
    Displays:
        * The animation in a pop-up window.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    def pause(duration):
        """
        Pauses the animation for duration seconds, provided duration > 0.
        """
        if duration > 0:
            plt.pause(duration)

    def tangent_line_at(x_0):
        return lambda x: f(x_0) + df(x_0) * (x - x_0)
    
    def iteration(x):
        return x - f(x) / df(x)

    cmap = plt.get_cmap("tab10")               # Used to control the colors.
    P = 200                                    # Number of points in each plot.
    width = 1.75                               # Line width.
    marker_size = 5
    domain = np.linspace(a, b, P)              # Generates P nodes from a to b.
    xs = [x]                                   # Stores the estimates.
    for _ in range(N):                         # Filling xs.
        xs.append(iteration(xs[-1]))
    ys = [f(x) for x in xs]                    # Stores f of the estimates.
    tangent_lines = [tangent_line_at(x) for x in xs]

    # Generate sample points for x-intervals between consecutive estimates:
    xs_range = [np.linspace(xs[n], xs[n + 1], P) for n in range(N)]
    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N)]

    # Draw the graph of f:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=width)
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.title(title)
    plt.grid(True)
    plt.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    plt.legend()

    # Mark the initial estimate on the x-axis:
    pause(duration)
    plt.plot(xs[0], 0, color=cmap(1), marker="x",
             mew=width, label=f'$ x_{0} $')

    for n in range(0, N):
        # Mark x_n on the x-axis:
        plt.plot(xs[n], 0, color=cmap(n + 1), marker="x", mew=width)
        pause(duration)
        # Draw the segment of the line x = x_n from y = 0 to y = y_n:
        plt.plot(xs_vert[n], ys_vert[n], linestyle='dotted',
                 lw=width, color='black')
        # Plot (x_n, y_n):
        plt.plot(xs[n], ys[n], color='black', marker="o", ms=marker_size)
        pause(duration)
        # Plot the tangent line at (x_n, f(x_n)):
        plt.plot(xs_range[n], tangent_lines[n](xs_range[n]),
                 linestyle='--', color=cmap(n + 2))
        pause(duration)
        plt.plot(xs[n + 1], 0, color=cmap(n + 2), marker="x",
                 mew=width, label=f'$ x_{n + 1} $')
        plt.legend()
    
    return xs, ys

In [6]:
from numpy import log
f = lambda x: x**3 - 2 * x - 3 * log(x)
df = lambda x: 3 * x**2 - 2 - 3 / x
a = 0.4
b = 1.5
x_0 = 1
N = 3
pausa = 1.0
titulo = "Exemplo: Método de Newton para $ y = x^3 - 2x - 3\,\ln x $."\
         "\n Estimativa inicial $ x_0 = 1 $ e $ N = 3 $ iterações."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         1.00000000        -1.00000000   |
|      01         0.50000000         1.20444154   |
|      02         0.66612987         0.18213330   |
|      03         0.70134213         0.00657080   |
|_________________________________________________|



![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_1.png
"Exemplo de aplicação do método de Newton")

__Problema 1:__ Usando o computador apenas como uma calculadora (e somente
quando necessário), aplique o método de Newton para aproximar um zero das
funções abaixo, usando $ N = 4 $ iterações e a estimativa inicial $ x_0 $
indicada.  Depois use os procedimentos `newton_animation` e `print_solution`
seguindo o modelo do Exemplo 1 para verificar sua resposta.

(a) $ y = xe^x - 1\, ,\quad  x_0 = 0 $. 

(b) $ y = \arctan x - 1\, ,\quad  x_0 = 1 $. 

(c) $ y = \ln x - 3\, ,\quad  x_0 = 10 $. 

(d) $ y = x^3 - 9x + 3\, ,\quad  x_0 = 3 $.

_Solução:_

__Problema 2:__ Sem usar o computador, encontre uma aproximação para $ \sqrt{2}
$  por um número racional com erro menor que $ 10^{-3} $ usando o método de
Newton.

_Solução:_

__Problema 3:__ Sem usar o computador, calcule uma aproximação com precisão de pelo menos dois dígitos decimais a uma raiz da equação $ x^3 - 2x - 16 $.

_Solução:_

## $ \S 2 $ Um critério suficiente para convergência do método de Newton

Suponha que $ f \colon J \to \mathbb R $ seja uma função diferenciável definida
num intervalo $ J $. Seja $ \alpha $ qualquer outra função diferenciável, mas
que _não se anula em $ J $_. Então, tomando
\begin{equation*}\label{E:special}
    \varphi(x) = x + \alpha(x)f(x)\,, \tag{1}
\end{equation*}
vale 
$$
\phantom{\qquad ( x \in I)}
f(x) = 0 \Longleftrightarrow \varphi(x) = x \qquad (x \in J)\,.
$$
Assim, em $ J $, os zeros de $ f $ são exatamente os pontos fixos de
$ \varphi $.

O fato que podemos transformar o problema de se encontrar zeros de $ f $ no
de se encontrar pontos fixos de outra função $ \varphi $ não é novo, já foi
discutido no caderno anterior. Lá foi visto que é possível garantir que o método
do ponto fixo aplicado a $ \varphi $ gera uma seqüência que converge a
um zero $ \zeta $ de $ f $ (ponto fixo de $ \varphi $) desde que tomemos a
estimativa inicial $ x_0 $ para $ \zeta $ dentro de um intervalo $ I \subset J $
centrado em $ \zeta $ onde valha
$$
    \vert \varphi'(x) \vert \le C \quad
    \text{para alguma constante $ C < 1 $ e todo $ x \in I $}\,.
$$
Como até agora não especificamos quem é a função $ \alpha $, podemos escolhê-la
da maneira mais conveniente possível, a saber, de modo que a derivada de
$ \varphi $ em $ \zeta $ seja nula:
$$
\varphi'(\zeta) = 1 + \alpha(\zeta) f'(\zeta) +
\alpha'(\zeta) \underbrace{f(\zeta)}_{0} = 0\,.
$$
Assumindo que $ f'(\zeta) \ne 0 $, gostaríamos portanto que
$$
    \alpha(\zeta) = -\frac{1}{f'(\zeta)}\,.
$$
Como $ \zeta $ é a princípio desconhecido, a maneira mais natural
de garantir que esta condição seja satisfeita é tomar
$$
    \alpha(x) = -\frac{1}{f'(x)}
$$
para todo $ x $ em $ J $ (desde que $ f'(x) \ne 0 $).
Fazendo esta escolha obtemos a função de iteração
\begin{equation*}\label{E:phi}
    \boxed{\varphi(x) = x -\frac{f(x)}{f'(x)}} \tag{2}
\end{equation*}
Concluímos que, dada uma estimativa inicial $ x_0 $, a seqüência $ (x_n) $
gerada através do método de Newton aplicado a $ f $ é exatamente a mesma que
aquela gerada pelo método do ponto fixo aplicado a esta função $ \varphi $.  Ou
seja, sob outro ponto de vista, o método de Newton surge de maneira natural como
caso especial do método do ponto fixo quando escolhemos uma função de iteração $
\varphi $ especialmente conveniente para o efeito de forçar a convergência a
$ \zeta $. Em particular, usando a teoria do método do ponto fixo deduzimos sem
esforço o seguinte resultado.

__Teorema 2.1 (critério suficiente para convergência do método de Newton):__
_Sejam $ f $ uma função duas vezes diferenciável e $ \zeta $ um zero de
$ f $ tal que $ f'(\zeta) \ne 0 $.  Suponha que $ I $ seja um intervalo centrado
em $ \zeta $ onde valha_
\begin{equation*}\label{E:criterion}
\boxed{
\left \lvert \frac{f\,f''}{\big(f'\big)^2} \right \rvert \tag{3}
\le C < 1}
\end{equation*}
_Então $ \zeta $ é o único zero de $ f $ em $ I $ e a seqüência $ (x_n) $ gerada
pelo método de Newton converge a $ \zeta $ para qualquer estimativa inicial
$ x_0 $ dentro de $ I $._

__Prova:__ Pela regra do quociente aplicada à função $ \varphi $ em \eqref{E:phi},
$$
\varphi'(x) = 1 - \frac{\big[f'(x)\big]^2 - f(x)f''(x)}{\big[f'(x)\big]^2}
= \frac{f(x)f''(x)}{\big[f'(x)\big]^2}\,.
$$
Logo a conclusão segue imediatamente do Corolário 6.2 do caderno anterior
contanto que seja satisfeita a sua hipótese, ou seja, que existam uma constante
positiva $ C < 1 $ e um intervalo $ I $ centrado em $ \zeta $ tais que 
$$
\left \lvert \varphi'(x) \right \rvert =
\left \lvert \frac{f(x)f''(x)}{\big[f'(x)\big]^2} \right \rvert
\le C \quad \text{para todo $ x \in I $}\,.
\tag*{$ \blacksquare $}
$$

Como conseqüência imediata obtemos o seguinte resultado importante.

__Corolário 2.2:__ _Sejam $ f $ uma função com segunda derivada contínua e
$ \zeta $ um zero de $ f $ tal que $ f'(\zeta) \ne 0 $. Então o método de Newton
aplicado a $ f $ gera uma seqüência convergente a $ \zeta $ para qualquer
estimativa inicial_ suficientemente próxima de $ \zeta $.

__Prova:__ Observe que a expressão à esquerda em \eqref{E:criterion} é
uma função contínua de $ x $ por hipótese e que ela se anula em $ x = \zeta $.
Em particular, existe um intervalo $ I $ centrado em $ \zeta $ onde ela é
$ \le \frac{1}{2} $.  Logo o Teorema 2.1 se aplica.

<div style="text-align: right">$ \blacksquare $ </div>

⚠️ Note que este critério é apenas _local_. Além disto, ele só pode ser aplicado
se o zero $ \zeta $ a ser aproximado não tiver multiplicidade maior que $ 1 $,
i.e., se $ f'(\zeta) \ne 0 $. 

📝 Para escolher a estimativa inicial o mais precisamente possível e assim
garantir a convergência, é recomendado analisar o gráfico da função antes de
aplicar o método de Newton.

__Problema 4:__

(a) Mostre que o método de Newton aplicado à função
$$
    \phantom{\qquad (x \in \mathbb R)} f(x) = \sqrt{1 + x^2} \qquad (x \in \mathbb R)
$$
com estimativa inicial $ x_0 = \frac{1}{2} $ produz uma seqüência periódica.

(b) É possível obter convergência trocando-se o valor de $ x_0 $? Justifique.

_Solução:_

__Problema 5:__ A função
$$
    \phantom{\qquad (x \in \mathbb R)} g(x) = \sqrt[3]{x} \qquad (x \in \mathbb R)
$$
possui um único zero $ \zeta = 0 $.

(a) Mostre que o método de Newton gera uma seqüência divergente para qualquer
estimativa inicial diferente de $ \zeta $.

(b) Explique por que isto não contradiz o Corolário 2.2.

_Solução:_

## $ \S 3 $ Vantagens e desvantages do método de Newton

### $ 3.1 $ Vantagens do método de Newton
Veremos abaixo que no método de Newton o erro cometido na iteração seguinte é
aproximadamente proporcional ao *quadrado* do erro da iteração atual. Portanto
podemos esperar que uma vez que o erro seja menor que $ 0.1 $, a cada passo o
número de dígitos decimais de precisão aproximadamente dobrará. 

Este desempenho deve ser comparado com o do método da bisseção, em que o erro
seguinte é aproximadamente proporcional ao erro atual (por um fator de $ 1/2 $).
Informalmente, isto significa que uma vez que consigamos uma estimativa próxima
o suficiente de um zero, o método de Newton convergirá _muito_ mais rapidamente
que os outros métodos que estudamos.

Outra vantagem é que para aplicar o método de Newton, não precisamos encaixotar
o zero.

### $ 3.2 $ Desvantagens do método de Newton

O método de Newton tem duas desvantagens significativas:
* Ele exige o cálculo da derivada da função à qual será aplicado.
* Ele nem sempre funciona, ou seja, a seqüência $ (x_n) $ pode não convergir.

Por causa do segundo defeito, uma estratégia mais adequada é utilizar o método
de Newton em combinação com um mais confiável, como o método da bissecção.
Começando com um intervalo $ [a, b] $ que encaixota um zero, utilizamos o método
de Newton começando com a estimativa inicial $ x_0 = \frac{a + b}{2} $. Mas
sempre que ocorrer
$$ \left\vert f(x_{n + 1}) \right\vert > \vert f(x_n) \vert \quad
\text{ou que} \quad x_{n + 1} \not \in [a, b]\,, $$
rejeitamos esta nova estimativa fornecida pelo método de Newton e aplicamos em
vez disto uma iteração do método da bissecção. Se as hipóteses do Corolário 2.2
são satisfeitas, eventualmente $ x_n $ ficará próximo o suficiente do zero que
está sendo aproximado, e então a convergência será rápida. Deixamos a cargo do
leitor implementar o procedimento descrito.

## $ \S 4 $ Dificuldades na convergência do método de Newton: estudo de casos

Vamos ilustrar através de alguns exemplos algumas dificuldades que podem surgir
na aplicação do método de Newton e que impedem a convergência ao zero desejado.

Grosso modo, as causas mais comuns para estas dificuldades são as seguintes:
* A estimativa inicial está distante demais do zero a ser aproximado.
* A derivada da função no zero não existe ou se anula.
* Existem pontos críticos na região onde o método
  está sendo aplicado.
* Não existe um zero na vizinhança de aplicação.

__Exemplo 2 (comportamento errático perto de pontos críticos):__
Recorde que $ a $ é dito um __ponto crítico__ de $ f $ se $ f'(a) \ne 0 $. Em
particular, qualquer extremo local (i.e., ponto de mínimo ou máximo) de uma
função diferenciável deve ser um ponto crítico.

Portanto, quando $ x_n $ está próximo de um extremo local, $
\left \vert f'(x_n) \right \vert $ é pequeno, logo a próxima estimativa
fornecida pode ficar muito distante da anterior. Nestes casos perdemos o
controle sobre o comportamento subseqüente. Além disto, o denominador
pequeno na fórmula de iteração amplifica erros de arredondamento e pode
até resultar num erro de _overflow_.

Esta situação está ilustrada na animação abaixo, em que o método de Newton
é aplicado à função
$$
    f(x) = 2x^5 - x + 1\,.
$$

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_2.png
"Exemplo de aplicação do método de Newton")

In [9]:
f = lambda x: 2 * x**5 - x + 1
df = lambda x: 10 * x**4 - 1
a = -1.1
b = 1.0
x_0 = 0.0
N = 4
pausa = 0.75
titulo = "Método de Newton para $ y = 2x^5 - x + 1 $. Perto do extremo"\
         "\n local, $ \\vert f' \\vert $ é pequeno; a estimativa é jogada para longe."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.00000000         1.00000000   |
|      01         1.00000000         2.00000000   |
|      02         0.77777778         0.79147826   |
|      03         0.48017397         0.57087924   |
|      04         1.69898988        27.61388409   |
|_________________________________________________|



__Exemplo 3 (a seqüência gerada pelo método converge a um zero distinto do desejado):__
A função
$$
    f(x) = \frac{1}{1 + x^2} - \frac{1}{2}
$$
tem derivada de todas as ordens e sua primeira derivada só se anula em
$ x = 0 $. Portanto, desde que ele seja evitado, não há obstáculos para se
aplicar o método de Newton para aproximar um dos dois zeros dela, em $ x = -1 $
e em $ x = 1 $. Contudo, se tomarmos como estimativa inicial $ x_0 = 2 $ com
intenção de aproximar o zero positivo, verificaremos que a seqüência resultante
acaba convergindo ao zero negativo. Em outras palavras, neste caso $ x_0 = 2 $
não está "próximo o suficiente" de $ 1 $ no sentido do Corolário 2.2.

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_3.png
"Exemplo de aplicação do método de Newton")

In [8]:
f = lambda x: 1 / (1 + x**2) - 1 / 2
df = lambda x: -2 * x * (1 + x**2)**(-2)
a = -2.2
b = 2.2
x_0 = 2
N = 7
pausa = 0.75
titulo = "Método de Newton para $ y = \\frac{1}{1 + x^2} - \\frac{1}{2} $"\
         " com $ x_0 = 2 $."\
         "\n A seqüência converge ao zero em $ x = -1 $, em vez de $ x = 1 $."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         2.00000000        -0.30000000   |
|      01         0.12500000         0.48461538   |
|      02         2.12451172        -0.31862865   |
|      03        -0.15508667         0.47651303   |
|      04        -1.76615596        -0.25724071   |
|      05        -0.53041070         0.28043572   |
|      06        -0.96443773         0.01809709   |
|      07        -0.99939056         0.00030481   |
|_________________________________________________|



__Exemplo 4 (a seqüência oscila entre dois valores sem convergir):__
Suponha que queiramos aproximar o zero $ \zeta = 0 $ da função seno. Seja $
\theta $ o menor ângulo positivo (em radianos) tal que
$$
    \tan(\theta) = 2\theta\,.
$$
Aplicando o método de Newton a $ f(x) = \sin x $ com estimativa inicial
$ x_0 = \theta $, obtemos como próxima estimativa
$$
x_1 = \theta - \frac{\sin \theta}{\cos \theta} =
\theta - \tan \theta = \theta - 2 \theta = -\theta \,.
$$
Agora, como a tangente é uma função ímpar, 
$$
x_2 = -\theta - \tan(-\theta) = -\theta + \tan(\theta)
= -\theta + 2\theta = \theta\,.
$$
Continuando, verifica-se por indução que as estimativas para o zero oscilam
entre os dois valores $ \pm \theta $.  Isto é ilustrado na animação abaixo. Note
que não há qualquer ponto crítico no intervalo de interesse. Entretanto, o zero
em $ \zeta = 0 $ é um ponto de inflexão (i.e., onde a segunda derivada
troca de sinal). Novamente, o problema aqui é que a estimativa inicial não é
próxima o suficiente do zero: se $ x_0 $ for tomado entre $ 0 $ e $ \theta $, a
seqüência correspondente converge a $ 0 $ como esperado. Contudo, se $ x_0 $ for
tomado um pouco maior que $ \theta $, o comportamento depende de maneira
extremamente sensível de $ x_0 $ (experimente, aumentando um pouco o valor de
$ x_0 $ na animação abaixo).

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_4.png
"Exemplo de aplicação do método de Newton")

In [9]:
from numpy import sin, cos
f = lambda x: sin(x)
df = lambda x: cos(x)
a = -1.4
b = 1.4
x_0 = 1.165561185207092    # Este é o ângulo θ mencionado no Exemplo.
N = 4
pausa = 0.75
titulo = "Método de Newton para $ y = \sin x $ com estimativa "\
         "inicial infeliz.\n As aproximações $ x_n $ oscilam entre "\
         "dois valores."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         1.16556119         0.91900972   |
|      01        -1.16556119        -0.91900972   |
|      02         1.16556119         0.91900972   |
|      03        -1.16556119        -0.91900972   |
|      04         1.16556119         0.91900972   |
|_________________________________________________|



__Exemplo 5 (a seqüência oscila entre mais de dois valores sem convergir):__
Generalizando o exemplo anterior, o método de Newton pode gerar seqüências
periódicas de qualquer período. No exemplo abaixo, ela oscila entre
exatamente três valores. O problema aqui é causado pela presença do mínimo local
em $ x = \frac{1}{\sqrt[4]{5}} $.

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_5.png
"Exemplo de aplicação do método de Newton")

In [9]:
f = lambda x: x**5 - x + 1
df = lambda x: 5 * x**4 - 1
a = -1.0
b = 1.0
N = 8
x_0 = -0.0833571     # Experimente alterar este valor!
pausa = 0.75
titulo = "Método de Newton para $ y = x^5 - x + 1 $ exibindo"\
         "\noscilação entre três valores."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00        -0.08335710         1.08335308   |
|      01         1.00025756         1.00103091   |
|      02         0.75032183         0.48749244   |
|      03        -0.08335710         1.08335308   |
|      04         1.00025756         1.00103091   |
|      05         0.75032183         0.48749244   |
|      06        -0.08335710         1.08335308   |
|      07         1.00025756         1.00103091   |
|      08         0.75032183         0.48749244   |
|_________________________________________________|




__Exemplo 6 (a convergência a um zero múltiplo pode ser lenta):__
A função $ f(x) = x^2 $ possui um zero de multiplicidade $ 2 $ em $ x = 0 $,
ou seja, não só vale $ f(0) = 0 $, mas também $ f'(0) = 0 $.
Neste caso, a iteração do método de Newton é dada por:
$$
x_{n + 1} = x_n - \frac{x_n^2}{2x_n} = \frac{x_n}{2}\,.
$$
Vemos portanto que os erros
$$
E_{n + 1} = 0 - x_{n + 1} \quad \text{e} \quad E_n = 0 - x_n
$$
cometidos na aproximação do zero por $ x_n $ são proporcionais por um fator de
$ \frac{1}{2} $. Isto significa que a convergência neste caso é tão rápida (ou
lenta) quanto a do método da bisseção.

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_6.png
"Exemplo de aplicação do método de Newton")

In [10]:
f = lambda x: x**2
df = lambda x: 2 * x
a = -1
b = 2
x_0 = 2
N = 5
pausa = 0.5
N = 4
titulo = "Método de Newton para $ y = x^2 $. Aproximação mais lenta"\
         "\na um zero múltiplo e problemas com o denominador pequeno."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         2.00000000         4.00000000   |
|      01         1.00000000         1.00000000   |
|      02         0.50000000         0.25000000   |
|      03         0.25000000         0.06250000   |
|      04         0.12500000         0.01562500   |
|_________________________________________________|



## $ \S 5 $ Implementação do método de Newton
 
Assim como no método do ponto fixo, a princípio não dispomos de um intervalo que
encaixota um zero nem de uma cota superior para o erro cometido que seja
independente da função. Por causa disto, não há um critério de parada
totalmente satisfatório. Na implementação abaixo, o procedimento é interrompido
assim que _ambas_ as desigualdades abaixo forem satisfeitas pela estimativa $
x_n $ atual:
\begin{equation*}\label{E:error}
\left\vert x_{n} - x_{n - 1} \right\vert < \varepsilon \quad \tag{4}
\end{equation*}
e
\begin{equation*}\label{E:magnitude}
\left\lvert f(x_n) \right\rvert < \varepsilon \,.\tag{5}
\end{equation*}
Como sempre, aqui $ \varepsilon > 0 $ é uma tolerância previamente escolhida.
Infelizmente, \eqref{E:error} não previne que ainda estejamos longe do zero,
pois esta quantidade pode ser pequena se a aproximação for muito lenta. E
\eqref{E:magnitude} tem o defeito de não ser invariante, no sentido que
apesar dos zeros das funções $ f(x) $ e $ kf(x) $ ($ k \ne 0 $ uma constante
real) serem exatamente os mesmos, o critério pode fornecer estimativas muito
diferentes se $ k $ está longe de $ 1 $.

Como a convergência do método de Newton não é garantida, é recomendável que o
procedimento seja interrompido caso um número máximo de iterações seja violado;
esta condição também está inclusa na implementação abaixo.

Outra dificuldade está relacionada ao denominador $ f'(x_n) $ que aparece na
fórmula para a próxima estimativa. Mesmo que a derivada $ f' $ exista em todo
ponto, se ela for muito pequena, isto pode gerar um erro de divisão por zero ou
de _overflow_.

In [5]:

def newton(f: Callable[[float], float], df: Callable[[float], float],
           x: float, eps: float = 1.0e-3, max_iter: int = 100
           ) -> tuple[list[float], list[float]]:
    """
    Applies Newton's method to try to find a zero of a function.
    Parameters:
        * A differentiable function f of one real variable.
        * Its derivative df (as another function).
        * An initial guess x for a zero.
        * A tolerance eps such that the procedure terminates if both
            |x_n - x_{n - 1}| < eps
          and
            |f(x_n)| < eps
          where x_n is the estimate provided by the current iteration.
        * The maximum allowed number of iterations, max_iter.
    Returns:
        * Two lists, xs and ys, containing the estimates and the values
          of f at each of them, respectively.
    Prints:
        * A warning or error message, in case of failure.
        * The last estimate.
        * The value of f at this estimate.
        * The number of iterations that were performed.
    """
    def iterate(x):
        """ Perform one step of Newton's method to yield the next estimate. """
        return x - f(x) / df(x)

    if eps <= 0:       # Error: invalid value for eps.
        raise ValueError("The tolerance must be positive!")
    # Check whether max_iter is a positive integer:
    if not (isinstance(max_iter, int) and max_iter > 0):
        raise ValueError("'max_iter' must be a positive integer!")
    x = float(x)       # Make sure x is of type float.
    xs = [x]           # Create a list to store the estimates.
    ys = [f(x)]        # List to store the values of f at the estimates.
    iterations = 0     # Counter for the number of iterations.
    error = 2 * eps    # Any value > eps will do.
    while error >= eps and iterations < max_iter:
        try:           # Compute the next estimate and check for overflow.
            new_x = iterate(x)
        except OverflowError:
            print("Either the estimates are diverging or the derivative"
                  " at the current estimate is too small.")
            return xs, ys
        else:
            xs.append(new_x)          # Store the new estimate.
            ys.append(f(new_x))       # Store f of the new estimate.
            # Update 'error':
            error = max(abs(xs[-1] - xs[-2]), abs(ys[-1])) 
            x = new_x                 # Update x.
            iterations += 1           # Update iteration counter.
    
    if iterations == max_iter:
        print("Warning: The maximum number of iterations was exceeded!")
    print(f"After {iterations} iterations, ", end='')
    print(f"the estimate for the fixed point is:\n{xs[-1]:15.8f}\n"
          f"The value of the function at this point is: \n{ys[-1]:15.8f}")
    return xs, ys

__Problema 6:__ Fazendo uso da implementação acima, aplique o método de Newton
para encontrar uma raiz das funções abaixo com erro menor que $ 10^{-5} $ nos
intervalos indicados:

(a) $ e^x + 2^{-x} + 2 \cos x = 6 $ em $ [1, 2] $.

(b) $ \cosh x \, \cos x = 1 $ em $ [4, 5] $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

(c) $ \tan x = \tanh x $ em $ [7.0, 7.4] $, onde por definição
$ \tanh x = \frac{\sinh x}{\cosh x} $ e $ \sinh x = \frac{e^x - e^{-x}}{2} $.

_Solução:_

__Problema 7:__ 

(a) Usando $ x_0 = 3 $ como estimativa inicial, encontre uma aproximação para
$ \pi $ com precisão de ao menos 7 dígitos decimais aplicando o método de Newton
para encontrar uma raiz das equações:
$$
\sin x = 0 \qquad \text{e} \qquad \cos x = -1\,.
$$

(b) Justifique a diferença no número de iterações necessárias.

_Solução:_

__Problema 8:__ Aproxime com precisão melhor que $ 10^{-5} $ o ponto sobre o
gráfico a função $ y = \frac{1}{x} $ ($ x > 0 $) que está à menor distância do
ponto $ P = (2, 1) $. _Dica:_ Seja $ d(x) $ a distância a $ P $ de um ponto
$ \big(x, \frac{1}{x}\big) $ genérico sobre este gráfico. Use o método de Newton
para encontrar os pontos onde a _derivada_ de $ x \mapsto d(x)^2 $ se anula.

_Solução:_

## $ \S 5 $ Análise do erro no método de Newton

### $ 5.1 $ Preliminares

__Teorema 5.1 (fórmula de Taylor de grau $ 1 $ com resto integral):__ _Sejam $ f $ uma função
com segunda derivada contínua num intervalo $ I $ e $ a \in I $. Então_
$$
f(x) = \underbrace{f(a) + f'(a)(x - a)}_{\substack{\text{polinômio de Taylor de grau $ 1 $} \\ \text{de $ f $ ao redor de $ a $}}} +
\underbrace{\int_a^x f''(t)(x - t)\,dt}_{\substack{\text{resto na forma
integral} \\ \text{(erro cometido na aproximação)}}} \qquad (x \in I)\,.
$$

__Prova:__ Pelo teorema fundamental do Cálculo vale
$$
f(x) = f(a) + \int_a^x f'(t)\,dt\,.
$$
Agora basta utilizar integração por partes para reescrever a integral. Tome
$$
\begin{cases}
    u = f'(t)\,, & du = f''(t)\,dt \\
    dv = dt\,, & v = -(x - t)
\end{cases}
$$
Então
\begin{alignat*}{9}
f(x) &= f(a) + \int_a^x f'(t)\,dt  \\
&= f(a) + \int_a^x u\,dv \\
&= f(a) + uv \big\vert_{t = a}^{t = x} - \int_a^x v\,du \\
&= f(a) -(x - t)f'(t) \big\vert_{t = a}^{t = x} + \int_a^x f''(t)(x - t)\,dt \\
&= f(a) + (x - a)f'(a) + \int_a^x f''(t)(x - t)\,dt\,. \tag*{$ \blacksquare $}
\end{alignat*}


__Teorema 5.2 (teorema do valor médio para integrais, versão estendida):__
_Sejam $ f,\, g \colon [a, b] \to \mathbb{R} $ funções contínuas tais que $ g $
não troca de sinal em $ [a, b] $. Então_ 
$$
\int_a^b f(x)\,g(x)\,dx = f(c)\int_a^bg(x)\,dx \qquad \text{para algum } c \in [a, b]\,.
$$

📝 Intuitivamente, na integral à esquerda, $ g $ tem o papel de uma função-peso. O
teorema diz que a "média ponderada"
$$
\frac{\int_a^b f(x)\,g(x)\,dx}{\int_a^bg(x)\,dx}
$$
de $ f $ é igual a $ f(c) $ para algum $ c $ entre $ a $ e $ b $. Tomando
$ g(x) \equiv 1 $ recuperamos o teorema "usual" do valor médio para integrais
em Cálculo.

__Prova:__ 
Sejam
$$
    m = \min_{[a, b]} f\quad \text{e} \quad M = \max_{[a, b]} f\,.
$$
Não há perda de generalidade ao se assumir que vale $ g \ge 0 $ em $ [a, b] $,
já que o caso em que $ g \le 0 $ pode ser reduzido a este trocando-se $ g $ por
$ -g $ (o que altera os sinais de ambos os lados da igualdade afirmada).
Então, sob esta hipótese, para qualquer $ x $ temos:
\begin{equation*}
    m\,g(x) \le f(x) g(x) \le M\, g(x)\,.
\end{equation*}
Integrando, deduzimos que
\begin{equation*}
m \int_a^b g(x)\,dx \le \int_a^b f(x)g(x)\,dx \le M \int_a^b g(x)\,dx \,.
\end{equation*}

Se $ g $ é identicamente nula, a igualdade afirmada é óbvia. Caso contrário,
a integral de $ g $ sobre $ [a, b] $ é estritamente positiva. Assim, dividindo
todos os termos da desigualdade acima por
$ \int_a^b g(x)\,dx $ concluímos que
\begin{equation*}
    m \le \frac{\int_a^b g(x)f(x)\,dx}{\int_a^b g(x)\,dx} \le M .
\end{equation*}
Como $ f $ é contínua por hipótese e $ m $ e $ M $ são seu mínimo e máximo
em $ [a, b] $ respectivamente, o teorema do valor intermediário garante a
existência de um $ c \in [a, b] $ tal que o termo do meio acima é igual a $ f(c)
$. Ou seja,
\begin{equation*}
    \int_a^b f(x)\,g(x)\,dx = f(c)\int_a^bg(x)\,dx
    \qquad \text{para algum } c \in [a, b]\,. \tag*{$ \blacksquare $}
\end{equation*}

### $ 5.2 $ Fórmula recursiva para o erro

Voltando ao estudo do método de Newton, seja $ x_n $ a $n$-ésima estimativa
para um zero $ \zeta $ de $ f $. Gostaríamos de obter uma fórmula relacionando
os erros
\begin{equation*}
E_{n + 1} = \zeta - x_{n + 1} \quad \text{e} \quad E_n = \zeta - x_n\,.
\end{equation*}

Pela fórmula de Taylor ao redor de
$ x_n $,
\begin{equation*}\label{E:zero}
f(\zeta) = 0 = f(x_n) + (\zeta - x_n)f'(x_n) +
\int_{x_n}^\zeta (\zeta - t)f''(t)\,dt\,. \tag{6}
\end{equation*}
Por outro lado, a próxima estimativa $ x_{n + 1} $ no método de Newton é definida
implicitamente pela igualdade 
\begin{equation*}\label{E:estimate}
0 = f(x_n) + (x_{n + 1} - x_n) f'(x_n) \tag{7}
\end{equation*}
Subtraindo \eqref{E:estimate} de \eqref{E:zero} deduzimos que
$$
0 = (\zeta - x_{n + 1})f'(x_n) +
\int_{x_n}^\zeta (\zeta - t)f''(t)\,dt\,.
$$
Agora, aplicando o Teorema 5.2 (com
$ g \leftarrow (\zeta - t) $ e $ f \leftarrow f'' $), concluímos
finalmente que
\begin{alignat*}{9}
0 &= (\zeta - x_{n + 1})f'(x_n) +
f''(c)\int_{x_n}^\zeta (\zeta - t)\,dt \\
&= \underbrace{(\zeta - x_{n + 1})}_{E_{n + 1}}\,f'(x_n) +
\frac{f''(c)}{2}\,\underbrace{(\zeta - x_n)^2}_{E_n}
\qquad \text{para algum $ c \in (a, b) $\,.}
\end{alignat*}
Daí deduzimos imediatamente a seguinte relação entre o erro atual $ E_n $ e o
próximo, $ E_{n + 1} $:
$$
\boxed{E_{n + 1} = -\frac{f''(c)}{2 f'(x_n)} E_n^2}
$$
Aqui $ c $ é algum ponto no menor intervalo contendo $ \zeta $ e $ x_n $.
A fórmula diz portanto que o novo erro é aproximadamente proporcional ao
_quadrado_ do anterior.

<div style="text-align: right">$ \blacksquare $ </div>

## $ \S 6 $ Problemas adicionais

__Problema 9:__ Fazendo referência ao Problema 2, suponha que queiramos
calcular $ \sqrt{2} $ através da aplicação do método de Newton à função
$ f(x) = x^2 - 2 $.

(a) Mostre que
$$
\frac{f(x)f''(x)}{f'(x)^2} = \frac{1}{2}\bigg(1 - \frac{2}{x^2}\bigg)\,.
$$

(b) Encontre $ r > 0 $ tal que o método de Newton aplicado a $ f $
gera uma seqüência convergente a $ \sqrt{2} $ desde que a estimativa
inicial $ x_0 $ seja tomada dentro do intervalo aberto de raio $ r $
e centro $ \sqrt{2} $; tome o maior $ r $ possível.
_Dica:_ Aplique o Teorema 2.1.


_Solução:_

__Problema 10:__ Seja $ a > 0 $. O método de Newton pode ser utilizado para se
calcular $ \frac{1}{a} $ sem efetuar qualquer operação de divisão! Alguns
computadores antigos utilizavam este método para implementar a divisão em
hardware de maneira eficiente.

(a) Mostre que a fórmula de iteração resultante da aplicação do método de
Newton a $ f(x) = a - \frac{1}{x} $ é
$$
x_{n + 1} = x_n \big(2 - ax_n \big)\,.
$$
Note que $ \frac{1}{a} $ é o único zero de $ f $.

(b) Usando Python apenas como uma calculadora, use este método para aproximar
$ \frac{1}{12} $ com as estimativas iniciais $ x_0 = 0.1 $ e $ x_0 = 1 $.
Compare os resultados.

(c) Seja $ r $ tal que $ 0 < r < \frac{1}{2a} $. Usando o Teorema 2.1, mostre
que a seqüência $ (x_n) $ resultante converge a $ \frac{1}{a} $ se $ x_0 $ for
tomado dentro do intervalo centrado em $ \frac{1}{a} $ de raio $ r $.

_Solução:_

__Problema 11:__ Seja $ r > 1 $ uma constante real. Compare a rapidez da
convergência dos métodos da bissecção e de Newton quando aplicados para
aproximar o zero $ \zeta = 0 $ da função definida por
$$
    \phantom{\qquad (x \in \mathbb R)} f_r(x) =
    \operatorname{sinal}(x)\, \vert x\vert ^r \qquad (x \in \mathbb R)\,.
$$
usando uma estimativa inicial qualquer. Justifique.

_Solução:_

__Problema 12:__ O objetivo deste problema é ilustrar que o Teorema 2.1 não
é _necessário_ para convergência do método de Newton a um zero. De fato,
o método pode funcionar mesmo quando a derivada da função não existe no zero.


(a) Seja $ r \in (0, 1) $. Mostre que
$$
\phantom{\qquad (x \in \mathbb R)}
f_r(x) = \operatorname{sinal}(x)\,\vert x\vert^r \qquad (x \in \mathbb R)\,,
$$
é diferenciável em todos os pontos, exceto no seu único zero $ \zeta = 0 $.
_Dica:_ Para $ x > 0 $, $ f(x) = x^r $ e $ f'(x) =rx^{r - 1} $. Para $ x < 0 $,
$ f(x) = -(-x)^{r} $, logo $ f'(x) = r(-x)^{r - 1} $. Portanto para qualquer
$ x \ne 0 $, $ f'(x) = r|x|^{r - 1} $. Conclua que a fórmula de iteração
fica $ x_{n + 1} = \frac{r - 1}{r} x_n $.

(b) Suponha agora que $ r \in \big(\frac{1}{2}, 1\big) $. 
Mostre que o método de Newton aplicado a $ f_r $ gera uma seqüência convergente
a $ \zeta = 0 $ para qualquer estimativa inicial $ x_0 $. _Dica:_
Compare $ \vert x_{n + 1} \vert $ com $ \vert x_n \vert $. 

(c) Por outro lado, mostre que se $ r \in \big(0, \frac{1}{2} \big) $, então
o método de Newton produz uma seqüência que diverge para qualquer estimativa
inicial distinta do zero $ \zeta = 0 $.

_Solução:_